In [45]:
import numpy as numpy
import pandas as pd
import json
import random
from sklearn.mixture import GaussianMixture
from keras.models import Sequential
from keras.layers import Dense

In [46]:
with open('reduced_dimension_stock_data.json') as json_file:
    stocks = json.load(json_file)

In [47]:
indList = []
#print(len(stocks))
for (key,value) in stocks.items():
    try:
        # print(value['Date'])
        cur = value['Date']
        indList.append(cur.index('2019-01-02', -900, -1))
        # cur = pd.DataFrame.from_dict(value['Date'])
        del value['Date']
    except Exception:
        indList.append(0)
        del value['Date']
# print(indList[0:100])

In [48]:
inputArray = pd.DataFrame.from_dict(stocks, orient='index')
#find longest sub-array
longest = 0
i=0
for (key, value) in stocks.items():
    if (len(inputArray.loc[key].at['Open']) > longest):
        longest = len(inputArray.loc[key].at['Open'])
    i+=1
#print("longest:")
#print(longest)
#in this dataset, longest is 14665
longest -= 365
i=0
err = 0
tick = {}
apple = None
for (key,value) in stocks.items():
    cur = inputArray.loc[key].at['Open']
    inputArray.at[key,'Open'] = cur[0:indList[i]]
    #duplicate first element to make array lengths match
    if key == 'AAPL':apple = i
    # print(inputArray.at[key,'Open'][0])
    try:
        firstEle = inputArray.at[key,'Open'][0]
        # print(firstEle, key)
        
        # print(firstEle != '0.0', firstEle)
        if (firstEle != '0.0'):
            cre = numpy.array([firstEle for j in range(longest - len(inputArray.at[key, 'Open']))])
            # print(cre)
            inputArray.at[key,'Open'] = numpy.append(cre,inputArray.at[key,'Open'])
            i+=1
            tick[i-err] = key
            # print(len(inputArray.at[key,'Open']) == longest)
        else:
            i+=1
            err+=1
            # print(i)
            # inputArray.drop(i,axis=0)
            inputArray = inputArray.drop(key)
            # i+=1
    except Exception:
        i+=1
        err+=1
        # print(i)
        # print('here')
        inputArray = inputArray.drop(key)
        # i+=1

In [49]:
# print(len(inputArray))
arr = pd.DataFrame.to_numpy(inputArray)
# print(numpy.shape(arr))
# print('sjhdgadjhlfgbadsv',arr[0][0]) 
newArr = numpy.zeros((694,longest))
# print(numpy.shape(arr), numpy.shape(arr[0]), numpy.shape(newArr))
for i in range(694):
    try:
        if(i == 616): continue
        newArr[i,:] = arr[i][0]
    except Exception:
        pass
#         print(i)

In [53]:
# difference = (newArr[:, -1] - newArr[:, 0]) / newArr[:, 0] ran into divide by zero YUCK
difference = (newArr[:, -1] - newArr[:, 0])
# print(difference.shape)
# print(difference)
ninetieth = numpy.percentile(difference, 90)
# print(ninetieth)
classification = difference > ninetieth
classification = classification.astype(int)
# print(classification)


In [55]:
model = Sequential()
model.add(Dense(500, input_dim = 14300, activation = 'relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

X = newArr
y = classification

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs=150, batch_size=10, verbose=0)

_, accuracy = model.evaluate(X, y)

print('Accuracy: %.2f' % (accuracy*100))

predictions = model.predict_classes(X)

recommendations = []

for i in range(694):
#     print('%d (expected %d)' % ( predictions[i], y[i]))
    if (predictions[i] == 1):
        recommendations.append(tick[i])

print("Recommendations: ")
print(recommendations)
# print(len(recommendations))
        
    

22/22 [==============================] - 0s 8ms/step - loss: 0.0595 - accuracy: 0.9827
Accuracy: 98.27
Recommendations: 
['AAP', 'ACN', 'ALB', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ATVI', 'AVB', 'AYX', 'BHF', 'BIIB', 'BKI', 'BLDR', 'CABO', 'CHDN', 'CHRW', 'CHTR', 'CME', 'COR', 'CRM', 'DOX', 'ESRT', 'EVR', 'FCX', 'FDS', 'FEYE', 'FIVN', 'GBT', 'GNRC', 'GOOG', 'GWRE', 'HCA', 'HIG', 'HUBS', 'HZNP', 'ICE', 'ICUI', 'INTC', 'IOVA', 'IRM', 'LECO', 'LHX', 'LYV', 'MET', 'MKSI', 'MKTX', 'MMC', 'MTCH', 'MTG', 'NFG', 'NVDA', 'ORCL', 'PODD', 'RCL', 'REG', 'ROKU', 'SAIC', 'SPG', 'STWD', 'TDC', 'TDOC', 'TPX', 'TSCO', 'UAL', 'UI', 'VRT', 'WAL']
